In [50]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

training_dataset = load_iris()
X_train, X_test, y_train, y_test = train_test_split(training_dataset['data'], training_dataset['target'], test_size=.3)

print("Training data shape", X_train.shape, y_train.shape)
print("Sample", X_train[0], y_train[0])

Training data shape (105, 4) (105,)
Sample [6.3 3.3 6.  2.5] 2


In [51]:
from xgboost import XGBClassifier
import mlflow.data
import mlflow
from sklearn.metrics import accuracy_score, f1_score, recall_score
from mlflow.models import infer_signature

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Iris on XGBoost")
# mlflow.autolog()

params = {
    'n_estimators': 5, 
    'max_depth': 50, 
    'learning_rate': .5, 
    'objective': 'binary:logistic'
}
training_dataset = mlflow.data.from_numpy(X_train, 'sklearn - iris', y_train)

with mlflow.start_run():
    bst = XGBClassifier(**params)
    bst.fit(X_train, y_train)
    mlflow.log_params(params)
    mlflow.set_tag('stage', 'training')
    mlflow.log_input(training_dataset, context='Training')

    metrics = {
        'accuracy': accuracy_score(y_test, bst.predict(X_test)),
        'recall': recall_score(y_test, bst.predict(X_test), average='macro'),
        'f1': f1_score(y_test, bst.predict(X_test), average='macro'),
    }
    mlflow.log_metrics(metrics)

    mlflow.xgboost.log_model(
        xgb_model=bst,
        artifact_path="iris_model",
        signature=infer_signature(X_train, y_train),
        input_example=X_train,
        registered_model_name="dev.xgb-iris.ml",
    )

c:\Users\yash.diniz\miniconda3\envs\mlflow\Lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'sklearn - iris'. Exception: 
  return _dataset_source_registry.resolve(
c:\Users\yash.diniz\miniconda3\envs\mlflow\Lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
c:\Users\yash.diniz\miniconda3\envs\mlflow\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:46:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose be

🏃 View run brawny-wolf-912 at: http://127.0.0.1:5000/#/experiments/610316935615800986/runs/874d456d33954627b24fe6acde9fe46f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/610316935615800986


In [52]:
client = mlflow.MlflowClient()
model_info = client.get_registered_model(f'dev.xgb-iris.ml')
client.set_registered_model_alias(f'dev.xgb-iris.ml', 'champion', model_info.latest_versions[0].version)

# client.delete_registered_model_tag(f'dev.xgb-iris.ml')

In [53]:
model = mlflow.xgboost.load_model(f"models:/dev.xgb-iris.ml@champion")

testing_dataset = mlflow.data.from_numpy(X_test, 'sklearn - iris', y_test)

with mlflow.start_run():
    mlflow.set_tag('stage', 'eval')
    preds = model.predict(X_test)
    mlflow.log_input(testing_dataset, context='Eval')

    cnt = 0
    for (i,k,v) in zip(range(0,len(X_test)), X_test, preds):
        print(k,v, y_test[i] == preds[i])
        if not(y_test[i] == preds[i]):
            cnt += 1

    print("error", cnt/len(X_test))

c:\Users\yash.diniz\miniconda3\envs\mlflow\Lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'sklearn - iris'. Exception: 
  return _dataset_source_registry.resolve(
c:\Users\yash.diniz\miniconda3\envs\mlflow\Lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(


[6.5 3.  5.8 2.2] 2 True
[6.  2.2 5.  1.5] 1 False
[6.8 2.8 4.8 1.4] 1 True
[4.8 3.4 1.9 0.2] 0 True
[4.6 3.6 1.  0.2] 0 True
[6.4 3.1 5.5 1.8] 2 True
[6.5 3.2 5.1 2. ] 2 True
[5.1 3.3 1.7 0.5] 0 True
[4.8 3.  1.4 0.3] 0 True
[7.1 3.  5.9 2.1] 2 True
[4.6 3.1 1.5 0.2] 0 True
[7.2 3.  5.8 1.6] 2 True
[5.8 2.7 5.1 1.9] 2 True
[5.8 2.7 5.1 1.9] 2 True
[5.4 3.4 1.7 0.2] 0 True
[6.4 3.2 4.5 1.5] 1 True
[4.9 2.4 3.3 1. ] 1 True
[5.2 3.5 1.5 0.2] 0 True
[6.  2.2 4.  1. ] 1 True
[6.7 3.3 5.7 2.5] 2 True
[4.9 3.  1.4 0.2] 0 True
[6.2 3.4 5.4 2.3] 2 True
[6.2 2.9 4.3 1.3] 1 True
[5.7 2.5 5.  2. ] 2 True
[5.6 2.7 4.2 1.3] 1 True
[6.1 2.9 4.7 1.4] 1 True
[6.6 3.  4.4 1.4] 1 True
[6.  3.  4.8 1.8] 2 True
[5.7 2.8 4.5 1.3] 1 True
[6.1 3.  4.9 1.8] 2 True
[6.2 2.2 4.5 1.5] 1 True
[5.  3.6 1.4 0.2] 0 True
[6.3 2.5 5.  1.9] 2 True
[7.  3.2 4.7 1.4] 1 True
[6.4 3.2 5.3 2.3] 2 True
[5.9 3.  4.2 1.5] 1 True
[5.5 2.6 4.4 1.2] 1 True
[6.1 2.8 4.7 1.2] 1 True
[5.1 2.5 3.  1.1] 0 False
[5.9 3.  5.1 1.8] 2 Tru